<a href="https://colab.research.google.com/github/s4kh4rov/BMIL/blob/master/lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
training_data = pd.read_excel('/content/drive/MyDrive/matches.xlsx')

In [50]:
training_data.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
1,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
2,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
3,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
4,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [51]:
training_data.shape #размеры загруженной таблицы

(1389, 28)

In [52]:
training_data.info() # вся информация, проверяем наличие пропусков и типы данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1389 entries, 0 to 1388
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1389 non-null   int64  
 1   date          1389 non-null   object 
 2   time          1389 non-null   object 
 3   comp          1389 non-null   object 
 4   round         1389 non-null   object 
 5   day           1389 non-null   object 
 6   venue         1389 non-null   object 
 7   result        1389 non-null   object 
 8   gf            1389 non-null   int64  
 9   ga            1389 non-null   int64  
 10  opponent      1389 non-null   object 
 11  xg            1389 non-null   float64
 12  xga           1389 non-null   float64
 13  poss          1389 non-null   float64
 14  attendance    693 non-null    float64
 15  captain       1389 non-null   object 
 16  formation     1389 non-null   object 
 17  referee       1389 non-null   object 
 18  match report  1389 non-null 

In [53]:
#приводим все типы object к numeric
training_data["date"] = pd.to_datetime(training_data["date"])
training_data["result"] = (training_data["result"] == "W").astype("int")
training_data["venue"] = training_data["venue"].astype("category").cat.codes
training_data["opponent"] = training_data["opponent"].astype("category").cat.codes
training_data["time"] = training_data["time"].str.replace(":.+", "", regex=True).astype("int")
training_data["comp"] = training_data["comp"].astype("category").cat.codes
training_data["round"] = training_data["round"].astype("category").cat.codes
training_data["day"] = training_data["day"].astype("category").cat.codes
training_data["captain"] = training_data["captain"].astype("category").cat.codes
training_data["formation"] = training_data["formation"].astype("category").cat.codes
training_data["referee"] = training_data["referee"].astype("category").cat.codes
training_data["match report"] = training_data["match report"].astype("category").cat.codes
training_data["team"] = training_data["team"].astype("category").cat.codes
#удалил пустые или не полностью заполненные столбцы
training_data.drop('notes', axis=1)
training_data.drop('attendance', axis=1)

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,1,2021-08-15,16,0,0,3,0,0,0,1,...,0,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,12
1,2,2021-08-21,15,0,11,2,1,1,5,0,...,0,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,12
2,3,2021-08-28,12,0,22,2,1,1,5,0,...,0,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,12
3,4,2021-09-11,15,0,32,2,0,1,1,0,...,0,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,12
4,6,2021-09-18,15,0,33,2,1,0,0,0,...,0,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,38,2021-05-02,19,0,27,3,0,0,0,4,...,0,NaN,8.0,1.0,17.4,0.0,0.0,0.0,2021,16
1385,39,2021-05-08,15,0,28,2,1,0,0,2,...,0,NaN,7.0,0.0,11.4,1.0,0.0,0.0,2021,16
1386,40,2021-05-16,19,0,29,3,0,1,1,0,...,0,NaN,10.0,3.0,17.0,0.0,0.0,0.0,2021,16
1387,41,2021-05-19,18,0,30,6,0,0,0,1,...,0,NaN,11.0,1.0,16.0,1.0,0.0,0.0,2021,16


In [55]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1389 entries, 0 to 1388
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1389 non-null   int64         
 1   date          1389 non-null   datetime64[ns]
 2   time          1389 non-null   int64         
 3   comp          1389 non-null   int8          
 4   round         1389 non-null   int8          
 5   day           1389 non-null   int8          
 6   venue         1389 non-null   int8          
 7   result        1389 non-null   int64         
 8   gf            1389 non-null   int64         
 9   ga            1389 non-null   int64         
 10  opponent      1389 non-null   int8          
 11  xg            1389 non-null   float64       
 12  xga           1389 non-null   float64       
 13  poss          1389 non-null   float64       
 14  attendance    693 non-null    float64       
 15  captain       1389 non-null   int8    

In [47]:
training_values = training_data['result'] # целевая переменная
training_points = training_data.drop('result', axis=1)

In [48]:
training_points.head()

,Unnamed: 0,date,time,comp,round,day,venue,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,1,2021-08-15,16,0,0,3,0,0,1,18,...,0,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,12
1,2,2021-08-21,15,0,11,2,1,5,0,15,...,0,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,12
2,3,2021-08-28,12,0,22,2,1,5,0,0,...,0,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,12
3,4,2021-09-11,15,0,32,2,0,1,0,10,...,0,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,12
4,6,2021-09-18,15,0,33,2,1,0,0,17,...,0,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,12


In [16]:
training_points.shape

(1389, 32)

In [ ]:
from sklearn import linear_model, ensemble
linear_regression_model = linear_model.LinearRegression()
random_forest_model = ensemble.RandomForestRegressor()

linear_regression_model.fit(training_points, training_values)